In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers

In [3]:


class_name1 = "person1"
class_name2 = "person2"
n_samples = 40

# Create directories for the classes
os.makedirs(class_name1, exist_ok=True)
os.makedirs(class_name2, exist_ok=True)

def generate_dataset(class_name):
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
   
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    cap = cv2.VideoCapture(0)
    img_id = 0

    while img_id < n_samples:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        cropped_face = face_cropped(frame)
        if cropped_face is not None:
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(f"{class_name}/img{img_id}.jpg", face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped_Face", face)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Collecting samples for {class_name} is completed !!!")

# Uncomment to generate dataset
generate_dataset(class_name1)
generate_dataset(class_name2)

def load_data(path):
    data = []
    for imgs in os.listdir(path):
        img = np.array(Image.open(f'{path}/{imgs}'))
        data.append(img)
    return data

def process_data(c1, c2):
    data = []
    x = []
    y = []
    for img in c1:
        data.append([img, [1, 0]])
    for img in c2:
        data.append([img, [0, 1]])
    for d in data:
        x.append(d[0])
    for d in data:
        y.append(d[1])
    return np.array(x), np.array(y)

def get_model(n_class):
    model = tf.keras.Sequential([
        tf.keras.Input((200, 200, 1)),
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(n_class, activation='softmax'),
    ])
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

class1, class2 = load_data(class_name1), load_data(class_name2)

x, y = process_data(class1, class2)

x = x / 255.0
x = x.reshape(x.shape[0], 200, 200, 1)
model = get_model(n_class=2)
model.fit(x, y, epochs=20)

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 558ms/step - accuracy: 0.5891 - loss: 9.9232
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 326ms/step - accuracy: 0.6008 - loss: 5.4622
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 308ms/step - accuracy: 0.5680 - loss: 4.0360
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 321ms/step - accuracy: 0.5242 - loss: 8.6197
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 330ms/step - accuracy: 0.4734 - loss: 7.8988
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 316ms/step - accuracy: 0.4766 - loss: 6.6951
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 322ms/step - accuracy: 0.4805 - loss: 7.2840
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 343ms/step - accuracy: 0.5000 - loss: 4.3330
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 341ms/step - accuracy: 0.5164 - loss: 2.7563
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 337ms/step - accuracy: 0.5281 - loss: 2.4909
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 363ms/step - accuracy: 0.5070 - loss: 3.3488
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 331ms/step - accuracy: 0.3977 - lo